In [ ]:
!pip install instaloader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 2.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
import instaloader

In [ ]:
test_data = pd.read_csv('test/datatest_revisi.csv')
train_data = pd.read_csv('train/datatrain.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      200 non-null    int64 
 1   video   200 non-null    object
dtypes: int64(1), object(1)
memory usage: 3.3+ KB


In [ ]:
train_data.head()

,id,video
0,1,https://www.instagram.com/reel/DM2DYuURXAS/?ig...
1,2,https://www.instagram.com/reel/DMpUrCKxQj5/?ig...
2,3,https://www.instagram.com/reel/DKoahGuRUGB/?ig...
3,4,https://www.instagram.com/reel/DKZVA8DJq89/?ig...
4,5,https://www.instagram.com/reel/DLtizEnyaOn/?ig...


In [ ]:
# for data train
train_data['emotion'].value_counts()

KeyError: 'emotion'

In [ ]:
import pandas as pd
from urllib.parse import urlparse

# Ambil domain dari kolom path
train_data["domain"] = train_data["video"].apply(lambda x: urlparse(str(x)).netloc)
test_data["domain"] = test_data["video"].apply(lambda x: urlparse(str(x)).netloc)

# Cek jumlah unik domain
print("Jumlah domain unik pada train:", train_data["domain"].nunique())
print("Daftar domain unik pada train:", train_data["domain"].unique())
print("Jumlah domain unik pada test:", test_data["domain"].nunique())
print("Daftar domain unik pada test:", test_data["domain"].unique())

# Hitung frekuensi tiap domain
print("\nFrekuensi domain:")
print(f"\nPada train {train_data["domain"].value_counts()}")
print(f"\nPada test {test_data["domain"].value_counts()}")

Jumlah domain unik: 3
Daftar domain unik: ['www.instagram.com' 'drive.google.com' 'instagram.com']

Frekuensi domain:
domain
www.instagram.com    140
drive.google.com      58
instagram.com          2
Name: count, dtype: int64


In [ ]:
print (f"Jumlah data train -> {train_data['video'].nunique()}")
print (f"\nJumlah data test -> {train_data['video'].nunique()}")

200

# Scrap Data

In [ ]:
!pip install gdown

In [ ]:
!pip install instagrapi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 2.2 MB/s eta 0:00:00


In [ ]:
!pip install instagrapi==1.17.10

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 12.0 MB/s eta 0:00:00
  Created wheel for instagrapi: filename=instagrapi-1.17.10-py3-none-any.whl size=103709 sha256=6c49974205ec01c5d72917f1914979255ed642291fac3151362419266568de11
  Stored in directory: /root/.cache/pip/wheels/60/8b/39/61a0930042ffd4b6760509263badbb1eb08296b28fe7f6604b
Successfully built instagrapi
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.7
    Uninstalling pydantic-2.11.7:
      Successfully uninstalled pydantic-2.11.7
  Attempting uninstall: pycryptodomex
    Found existing installation: pycryptodomex 3.23.0
    Uninstalling pycryptodomex-3.23.0:
      Successfully uninstalled pycryptodomex-3.2

In [ ]:
import os
import pandas as pd
import requests
import gdown
import shutil
from urllib.parse import urlparse
from instagrapi import Client

# === Konfigurasi ===
CSV_INPUT = "/content/drive/MyDrive/Satria_Data/test/datatest_revisi.csv"
CSV_OUTPUT = "/content/drive/MyDrive/Satria_Data/test/test_scrap.csv"
OUTPUT_DIR = "/content/drive/MyDrive/Satria_Data/test/videos"

# Buat folder output
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Baca CSV
df = pd.read_csv(CSV_INPUT)

# Setup Instagrapi client
cl = Client()

# ⚠️ Gunakan sessionid biar stabil
SESSIONID = "55201690344%3A5aGhKyhejxLaKE%3A20%3AAYcgsOq-V8biGFF5Qh5sbsSjBfhSDjB7WLZRYH5Gjw"  # ganti dengan sessionid dari cookie IG
cl.login_by_sessionid(SESSIONID)

# List hasil
new_data = []

# === Fungsi bantu ===

def download_instagram(url, output_path):
    """Download Instagram Reels dengan custom name"""
    try:
        media_pk = cl.media_pk_from_url(url)
        temp_path = cl.video_download(media_pk)

        # pakai shutil.move biar aman cross-device
        if temp_path != output_path:
            if os.path.exists(output_path):
                os.remove(output_path)  # hapus kalau sudah ada
            shutil.move(temp_path, output_path)

        return True
    except Exception as e:
        print(f"[IG FAIL] {url} - {e}")
        return False


def download_drive(url, output_path):
    """Download Google Drive file"""
    try:
        if "id=" in url:
            file_id = url.split("id=")[-1]
        else:
            file_id = url.split("/d/")[1].split("/")[0]
        gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)
        return True
    except Exception as e:
        print(f"[Drive FAIL] {url} - {e}")
        return False


def download_cdn(url, output_path):
    """Download langsung dari CDN Instagram"""
    try:
        r = requests.get(url, stream=True)
        with open(output_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
        return True
    except Exception as e:
        print(f"[CDN FAIL] {url} - {e}")
        return False


# === Proses setiap baris ===
for _, row in df.iterrows():
    vid_id = row["id"]
    url = row["video"]

    output_path = os.path.join(OUTPUT_DIR, f"{vid_id}.mp4")

    if os.path.exists(output_path):
        print(f"[SKIP] {output_path} sudah ada")
        new_data.append({"id": vid_id, "video": output_path})
        continue

    domain = urlparse(url).netloc
    success = False

    if "instagram.com" in domain and "scontent" not in domain and "fna.fbcdn.net" not in domain:
        print(f"[Instagram] Download {url}")
        success = download_instagram(url, output_path)

    elif "drive.google.com" in domain:
        print(f"[Google Drive] Download {url}")
        success = download_drive(url, output_path)

    elif "scontent" in domain or "fna.fbcdn.net" in domain:
        print(f"[CDN] Download {url}")
        success = download_cdn(url, output_path)

    else:
        print(f"[UNKNOWN] {url} -> dilewati")

    if success:
        new_data.append({"id": vid_id, "video": output_path})
        print(f"[OK] {output_path}")
    else:
        new_data.append({"id": vid_id, "video": None})
        print(f"[FAIL] {url}")

# Simpan CSV baru
new_df = pd.DataFrame(new_data)
new_df.to_csv(CSV_OUTPUT, index=False)

print(f"\nCSV baru disimpan sebagai {CSV_OUTPUT}")

[SKIP] /content/drive/MyDrive/Satria_Data/test/videos/1.mp4 sudah ada
[SKIP] /content/drive/MyDrive/Satria_Data/test/videos/2.mp4 sudah ada
[SKIP] /content/drive/MyDrive/Satria_Data/test/videos/3.mp4 sudah ada
[SKIP] /content/drive/MyDrive/Satria_Data/test/videos/4.mp4 sudah ada
[SKIP] /content/drive/MyDrive/Satria_Data/test/videos/5.mp4 sudah ada
[SKIP] /content/drive/MyDrive/Satria_Data/test/videos/6.mp4 sudah ada
[SKIP] /content/drive/MyDrive/Satria_Data/test/videos/7.mp4 sudah ada
[SKIP] /content/drive/MyDrive/Satria_Data/test/videos/8.mp4 sudah ada
[SKIP] /content/drive/MyDrive/Satria_Data/test/videos/9.mp4 sudah ada
[SKIP] /content/drive/MyDrive/Satria_Data/test/videos/10.mp4 sudah ada
[SKIP] /content/drive/MyDrive/Satria_Data/test/videos/11.mp4 sudah ada
[SKIP] /content/drive/MyDrive/Satria_Data/test/videos/12.mp4 sudah ada
[SKIP] /content/drive/MyDrive/Satria_Data/test/videos/13.mp4 sudah ada
[SKIP] /content/drive/MyDrive/Satria_Data/test/videos/14.mp4 sudah ada
[SKIP] /content

Downloading...
From: https://drive.google.com/uc?id=1-OawfKdJ7ozcradDR4rxu4QCWtd93qY5
To: /content/drive/MyDrive/Satria_Data/test/videos/58.mp4
100%|██████████| 6.32M/6.32M [00:00<00:00, 72.0MB/s]


[OK] /content/drive/MyDrive/Satria_Data/test/videos/58.mp4
[Google Drive] Download https://drive.google.com/file/d/1UI8IJ19DQkZuTaUtLVtAI19jGMIDJuO_/view?usp=sharing


Downloading...
From: https://drive.google.com/uc?id=1UI8IJ19DQkZuTaUtLVtAI19jGMIDJuO_
To: /content/drive/MyDrive/Satria_Data/test/videos/59.mp4
100%|██████████| 6.71M/6.71M [00:00<00:00, 63.8MB/s]


[OK] /content/drive/MyDrive/Satria_Data/test/videos/59.mp4
[Google Drive] Download https://drive.google.com/file/d/1nbo5CnW50clco9ZFd6AE4R6KUwPTYpLA/view?usp=share_link


...

[OK] /content/drive/MyDrive/Satria_Data/test/videos/189.mp4
[Google Drive] Download https://drive.google.com/file/d/1NIWRtQwRup1CXoykRSIJBBAvrMr-Ochv/view?usp=drive_link


Downloading...
From: https://drive.google.com/uc?id=1NIWRtQwRup1CXoykRSIJBBAvrMr-Ochv
To: /content/drive/MyDrive/Satria_Data/test/videos/190.mp4
100%|██████████| 11.1M/11.1M [00:00<00:00, 28.1MB/s]


[OK] /content/drive/MyDrive/Satria_Data/test/videos/190.mp4
[Google Drive] Download https://drive.google.com/file/d/1DSfKv-iiVhftQ6Cz3v_qu7pop9WfQvWO/view?usp=drive_link


Downloading...
From: https://drive.google.com/uc?id=1DSfKv-iiVhftQ6Cz3v_qu7pop9WfQvWO
To: /content/drive/MyDrive/Satria_Data/test/videos/191.mp4
100%|██████████| 3.56M/3.56M [00:00<00:00, 32.3MB/s]


[OK] /content/drive/MyDrive/Satria_Data/test/videos/191.mp4
[Instagram] Download https://www.instagram.com/reel/DCYxVATvxDu/?igsh=bTI4cHhqajE5ZWE5
[OK] /content/drive/MyDrive/Satria_Data/test/videos/192.mp4
[Instagram] Download https://www.instagram.com/reel/DCrH5IdzQCj/?igsh=aHAwbmN2amJqeXpy
[OK] /content/drive/MyDrive/Satria_Data/test/videos/193.mp4
[Instagram] Download https://www.instagram.com/reel/DMzjk18xmcY/?utm_source=ig_web_copy_link
[OK] /content/drive/MyDrive/Satria_Data/test/videos/194.mp4
[Instagram] Download https://www.instagram.com/reel/DM-JaK4yddh/?utm_source=ig_web_copy_link
[OK] /content/drive/MyDrive/Satria_Data/test/videos/195.mp4
[Instagram] Download https://www.instagram.com/reel/DMUQW-dy0Nm/?utm_source=ig_web_copy_link&igsh=MzRlODBiNWFlZA==
[OK] /content/drive/MyDrive/Satria_Data/test/videos/196.mp4
[Instagram] Download https://www.instagram.com/reel/DL4bTmcS4r8/?utm_source=ig_web_copy_link&igsh=MzRlODBiNWFlZA==
[OK] /content/drive/MyDrive/Satria_Data/test/video

In [ ]:
# test download
import os
import pandas as pd
import requests
import gdown
import shutil
from urllib.parse import urlparse
from instagrapi import Client

# === Konfigurasi ===
CSV_INPUT = "/content/drive/MyDrive/Lomba/Satria_Data/test/datatest.csv"
CSV_OUTPUT = "/content/drive/MyDrive/Lomba/Satria_Data/test/test_scrap.csv"
OUTPUT_DIR = "/content/drive/MyDrive/Lomba/Satria_Data/test/videos"

# Buat folder output
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Baca CSV
df = pd.read_csv(CSV_INPUT)

# Setup Instagrapi client
cl = Client()

# ⚠️ Gunakan sessionid biar stabil
SESSIONID = "55201690344%3A5aGhKyhejxLaKE%3A20%3AAYcgsOq-V8biGFF5Qh5sbsSjBfhSDjB7WLZRYH5Gjw"  # ganti dengan sessionid dari cookie IG
cl.login_by_sessionid(SESSIONID)

# List hasil
new_data = []

# === Fungsi bantu ===

def download_instagram(url, output_path):
    """Download Instagram Reels dengan custom name"""
    try:
        media_pk = cl.media_pk_from_url(url)
        temp_path = cl.video_download(media_pk)

        # pakai shutil.move biar aman cross-device
        if temp_path != output_path:
            if os.path.exists(output_path):
                os.remove(output_path)  # hapus kalau sudah ada
            shutil.move(temp_path, output_path)

        return True
    except Exception as e:
        print(f"[IG FAIL] {url} - {e}")
        return False


def download_drive(url, output_path):
    """Download Google Drive file"""
    try:
        if "id=" in url:
            file_id = url.split("id=")[-1]
        else:
            file_id = url.split("/d/")[1].split("/")[0]
        gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)
        return True
    except Exception as e:
        print(f"[Drive FAIL] {url} - {e}")
        return False


def download_cdn(url, output_path):
    """Download langsung dari CDN Instagram"""
    try:
        r = requests.get(url, stream=True)
        with open(output_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
        return True
    except Exception as e:
        print(f"[CDN FAIL] {url} - {e}")
        return False


# === Proses setiap baris ===
for _, row in df.iterrows():
    vid_id = row["id"]
    emotion = row["emotion"]
    url = row["videos"]

    output_path = os.path.join(OUTPUT_DIR, f"{vid_id}_{emotion}.mp4")

    if os.path.exists(output_path):
        print(f"[SKIP] {output_path} sudah ada")
        new_data.append({"id": vid_id, "path": output_path, "emotion": emotion})
        continue

    domain = urlparse(url).netloc
    success = False

    if "instagram.com" in domain and "scontent" not in domain and "fna.fbcdn.net" not in domain:
        print(f"[Instagram] Download {url}")
        success = download_instagram(url, output_path)

    elif "drive.google.com" in domain:
        print(f"[Google Drive] Download {url}")
        success = download_drive(url, output_path)

    elif "scontent" in domain or "fna.fbcdn.net" in domain:
        print(f"[CDN] Download {url}")
        success = download_cdn(url, output_path)

    else:
        print(f"[UNKNOWN] {url} -> dilewati")

    if success:
        new_data.append({"id": vid_id, "path": output_path, "emotion": emotion})
        print(f"[OK] {output_path}")
    else:
        new_data.append({"id": vid_id, "path": None, "emotion": emotion})
        print(f"[FAIL] {url}")

# Simpan CSV baru
new_df = pd.DataFrame(new_data)
new_df.to_csv(CSV_OUTPUT, index=False)

print(f"\nCSV baru disimpan sebagai {CSV_OUTPUT}")